# Keras Self-Define自定义

## 自定义loss函数
一定要写成`y_true, y_pred`

In [1]:
from keras import backend as K

def MSE_loss(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

Using TensorFlow backend.


## 自定义metrics指标
与定义loss函数相似，一定要写成`y_true, y_pred`

In [2]:
from keras import backend as K

def categorical_accuracy(y_true, y_pred):
    return K.cast(K.equal(K.argmax(y_true, axis=-1),
                          K.argmax(y_pred, axis=-1)),
                  K.floatx())

## 自定义layer
以全相连网络为例。<br>
以下的是一些基础参数，必须要有，其他的可以自行添加。<br>
`bias`中的`shape`一定要写成这样`shape=(self.output_dim,)`。这是一个迭代器。

In [3]:
from keras import backend as K
from keras.engine.topology import Layer

class Linear(Layer):
    def __init__(self, output_dim,
                 **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.output_dim = output_dim

    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        self.kernel = self.add_weight(
            name='weights',
            shape=(input_dim, self.output_dim),
            initializer='uniform',
            trainable=True)
        self.bias = self.add_weight(
            name='bias',
            shape=(self.output_dim,),
            initializer='uniform',
            trainable=True)

    def call(self, x):
        out = K.dot(x, self.kernel)
        out = K.bias_add(out, self.bias)
        return out

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

## 自定义 optimizer

In [4]:
from keras import backend as K
from keras.optimizers import Optimizer


class toy_SGD(Optimizer):
    def __init__(self, lr=0.01, **kwargs):
        super(toy_SGD, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        lr = self.lr
        for p, g in zip(params, grads):
            v = - lr * g  # velocity
            new_p = p + v
            self.updates.append(K.update(p, new_p))
        return self.updates